In [1]:
import dolfinx
import numpy as np
import ufl
import pyvista

### Formulating the variational problem

In [2]:
from mpi4py import MPI # something something message passing system, for communication?
from dolfinx import mesh, fem, io, plot, default_scalar_type


# importing a built-in mesh 
# a uniform square mesh of 8x8 with quadrilateral cells 
# MPI tells you how to distribute the mesh data over your processors, that's relevant for later
domain = mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, mesh.CellType.quadrilateral)


# create a FE function space V with the space initialiser dolfinx.fem
fspace = fem.functionspace
# the finite element family (Lagrange elements) and polynomial degree (1)
V = fspace(domain, ("Lagrange", 1)) 

# set boundary data in the discrete trial space
uD = fem.Function(V)
uD.interpolate(lambda x: 1+ x[0]**2 + 2 * x[1]**2)

# apply boundary values to all dofs that are on the boundary of the discrete domain
# facets in 3D, line-segments in 2D
tdim = domain.topology.dim # dimensionality of the domain (mesh) is stored here
fdim = tdim -1 # facet dimension, one less than the mesh dim

# finds the outer boundary facets and stores them
domain.topology.create_connectivity(fdim, tdim) # so mapping from 1D to 2D
boundary_facets = mesh.exterior_facet_indices(domain.topology)
print(boundary_facets)

# finding the local indices of the degrees of freedom and setting a Dirichlet boundary condition
# Dirichlet = fixed solution values at the boundary of the domain
boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs) # value = uD, dofs = boundary_dofs, V = V

# print(boundary_dofs)


[  0   1   3   5   9  13  17  23  27  35  39  49  53  65  69  83  86  87
 100 101 102 113 114 123 124 131 132 137 138 141 142 143]


In [3]:
# notations as in the tutorial
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
f = fem.Constant(domain, default_scalar_type(-6))
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx 
L = f * v * ufl.dx

### Solving the problem

In [4]:
from dolfinx.fem.petsc import LinearProblem

# lu is the solver, ksp_type is smt.
problem = LinearProblem(a, L, bcs = [bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve() # the discrete solution uh as defined in the theory

# computing the error in two ways
V2 = fem.functionspace(domain, ("Lagrange", 2))
uex = fem.Function(V2)
uex.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)

# L2 norm error - basically how much your trial func at the boundary 
# differs from the actual boundary condition
L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = np.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))

# errors on the dofs
error_max = np.max(np.abs(uD.x.array-uh.x.array))

# Only print the error on one process
if domain.comm.rank == 0:
    print(f"Error_L2 : {error_L2:.2e}")
    print(f"Error_max : {error_max:.2e}")

Error_L2 : 8.24e-03
Error_max : 3.55e-15


## Plotting the outcomes

In [5]:
# pyvista.set_jupyter_backend(html)
print(pyvista.global_theme.jupyter_backend)
pyvista.start_xvfb()
domain.topology.create_connectivity(tdim, tdim)

# converting the mesh to a format that can be used with pyvista
# we create a mesh with 
# note to self: domain is prob. what we want to plot, see if I can load it into another visualizer
topology, cell_types, geometry = plot.vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

print(grid)

# pyvista.Report(gpu=True)

trame
UnstructuredGrid (0x7b478d9c95a0)
  N Cells:    64
  N Points:   81
  X Bounds:   0.000e+00, 1.000e+00
  Y Bounds:   0.000e+00, 1.000e+00
  Z Bounds:   0.000e+00, 0.000e+00
  N Arrays:   0


Mesh plot

In [6]:
plotter = pyvista.Plotter(notebook=True)
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
#figure = plotter.screenshot("fundamentals_mesh.png")
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("fundamentals_mesh.png")

Widget(value='<iframe src="http://localhost:35893/index.html?ui=P_0x7b4667005e80_0&reconnect=auto" class="pyvi…